# Policy Impact Analysis
This notebook iterates through 6 different policy reforms and calculates their 10-year budgetary impacts.

In [9]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

## Define Policy Reform Functions
Each function returns a Reform object for different policy options

In [ ]:
# Option 1: Tax Credit Expansion
def get_repeal_social_security_reform():
    # Placeholder for Tax Credit Expansion reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        }
        }, country_id="us")

# Policy 2: Infrastructure Investment
def get_flat_ss_tax_reform():
    # Placeholder for Infrastructure Investment reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2024-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2100-12-31": 0
        }
        }, country_id="us")

# Policy 3: Healthcare Reform
def get_flat_ss_tax_with_senior_deduction_extension_reform():
    # Placeholder for Healthcare Reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        # Add your reform parameters here
    }, country_id="us")

# Policy 4: Child Tax Credit (with variable amounts)
def get_ss_credit_reform(amount):
    return Reform.from_dict({
        "gov.contrib.crfb.ss_credit.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.ss_credit.amount.SINGLE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.JOINT": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
    }, country_id="us")

# Policy 5: Education Funding
def get_roth_style_swap_reform():
    # Placeholder for Education Funding reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2025-01-01.2100-12-31": True
        }
        }, country_id="us")

# Policy 6: Green Energy Incentives
def get_roth_style_swap_with_phase_in_reform():
    # Placeholder for Green Energy Incentives reform
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        # Add your reform parameters here
    }, country_id="us")

## Calculate Impact for Each Year
Function to calculate the budgetary impact for a given reform and year

In [11]:
def calculate_yearly_impact(reform, year):
    """
    Calculate the budgetary impact for a given reform and year.
    Returns impact in billions (negative = cost, positive = savings)
    """
    baseline = Microsimulation()
    baseline_income = baseline.calculate("household_net_income", period=year)
    
    reformed = Microsimulation(reform=reform)
    reformed_income = reformed.calculate("household_net_income", period=year)
    difference_income = reformed_income - baseline_income
    impact_billions = difference_income.sum() / 1e9
    
    return round(impact_billions, 1)


## Process All 6 Policy Reforms
Iterate through each policy and calculate 10-year impacts

In [ ]:
# Define years to analyze
years = list(range(2025, 2035))

# Initialize results list
all_results = []

# Policy configurations
policies = [
    {
        'id': 1,
        'name': 'Tax Credit Expansion',
        'reform_func': get_repeal_social_security_reform,
        'has_variants': False
    },
    {
        'id': 2,
        'name': 'Flat Social Security Tax',
        'reform_func': get_flat_ss_tax_reform,
        'has_variants': False
    },
    {
        'id': 3,
        'name': 'Flat Social Security Tax with Bonus Senior Deduction extension',
        'reform_func': get_flat_ss_tax_with_senior_deduction_extension_reform,
        'has_variants': False
    },
    {
        'id': 4,
        'name': 'Social Security Credit',
        'reform_func': get_ss_credit_reform,
        'has_variants': True,
        'variants': [300, 600, 900, 1200, 1500]
    },
    {
        'id': 5,
        'name': 'Roth Style Swap',
        'reform_func': get_roth_style_swap_reform,
        'has_variants': False
    },
    {
        'id': 6,
        'name': 'Roth Style Swap with Phase-In',
        'reform_func': get_roth_style_swap_with_phase_in_reform,
        'has_variants': False
    }
]

In [13]:
# Process each policy
for policy in policies:
    print(f"\nProcessing Policy {policy['id']}: {policy['name']}")
    
    if policy['has_variants']:
        # Handle policies with multiple variants (e.g., Child Tax Credit)
        for variant in policy['variants']:
            print(f"  Variant: ${variant}")
            reform = policy['reform_func'](variant)
            
            for year in years:
                impact = calculate_yearly_impact(reform, year)
                all_results.append({
                    'policy_id': policy['id'],
                    'policy_name': policy['name'],
                    'credit_value': variant,
                    'year': year,
                    'impact_billions': impact
                })
    else:
        # Handle standard policies
        try:
            reform = policy['reform_func']()
        except:
            # Use None if reform function not implemented
            reform = None
            print(f"  Note: Using placeholder values (reform not yet implemented)")
        
        for year in years:
            impact = calculate_yearly_impact(reform, year)
            all_results.append({
                'policy_id': policy['id'],
                'policy_name': policy['name'],
                'credit_value': None,
                'year': year,
                'impact_billions': impact
            })

print("\nProcessing complete!")


Processing Policy 1: Tax Credit Expansion


KeyboardInterrupt: 

## Create and Save Results DataFrame

In [ ]:
# Create DataFrame from results
df_results = pd.DataFrame(all_results)

# Display summary statistics
print("\n=== Summary Statistics ===")
print(f"Total records: {len(df_results)}")
print(f"\nPolicies processed:")
for policy_id, group in df_results.groupby('policy_id'):
    policy_name = group['policy_name'].iloc[0]
    if group['credit_value'].notna().any():
        variants = group['credit_value'].dropna().unique()
        print(f"  {policy_id}. {policy_name} - {len(variants)} variants")
    else:
        print(f"  {policy_id}. {policy_name}")

# Display first few rows
print("\n=== First 10 rows ===")
df_results.head(10)

In [ ]:
# Save to CSV file
output_file = 'policy_impacts.csv'
df_results.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

# Also save to the React app's public folder
react_output = 'policy-impact-dashboard/public/policy_impacts.csv'
df_results.to_csv(react_output, index=False)
print(f"Results also saved to {react_output}")